# 2.3 Data Aggregation

In this section, you will learn how to:

*   Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)

*   Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function

*   Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection

*   Compute pivot tables and cross-tabulations

*   Perform quantile analysis and other statistical group analyses


source: [McKinney, Wes. Python for Data Analysis. O'Reilly Media. 3rd Edition](https://wesmckinney.com/book/data-aggregation)

## 2.3.1 How to Think About Group Operations

***split-apply-combine*  for group operations.**

*   In the first stage of the process, data contained in a pandas object is *split* into groups based on one or more keys that we provide. The *splitting* is performed on a particular axis of an object. 
*   Next, a function is *applied* to each group, producing a new value.
*   Finally, the results of all those function applications are *combined* into a result object.  


In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(12345)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [3]:
df.shape

(7, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   key1    5 non-null      object 
 1   key2    6 non-null      Int64  
 2   data1   7 non-null      float64
 3   data2   7 non-null      float64
dtypes: Int64(1), float64(2), object(1)
memory usage: 359.0+ bytes


In [5]:
grouped1 = df.groupby(df["key1"])
grouped1

In [6]:
# print out the groups 
for group_key, group_values in grouped1:
    print(group_key)
    print(group_values)

a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [7]:
grouped1["data1"].describe().round(6)

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.555881,0.801830,-0.204708,0.137118,0.478943,0.936175,1.393406
b,2.0,0.705025,1.782977,-0.555730,0.074647,0.705025,1.335403,1.965781


In [8]:
grouped1["data2"].describe().round(6)

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.441920,0.283299,0.274992,0.278369,0.281746,0.525384,0.769023
b,2.0,-0.144516,1.628757,-1.296221,-0.720368,-0.144516,0.431337,1.007189


In [9]:
grouped1.sum()

,key2,data1,data2
key1,,,
a,3,1.667642,1.325760
b,3,1.410050,-0.289032


In [10]:
grouped1.sum().unstack()

       key1
key2   a              3
       b              3
data1  a       1.667642
       b        1.41005
data2  a        1.32576
       b      -0.289032
dtype: object

In [11]:
###  grouping the data using two keys.
grouped2 = df.groupby([df["key1"], df["key2"]])
grouped2

In [12]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [13]:
# print out the groups 
for group_key, group_values in grouped2:
    print(group_key,":")
    print(group_values)

('a', 1) :
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
('a', 2) :
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
('b', 1) :
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
('b', 2) :
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


In [14]:
grouped2["data1"].describe().round(6)

count      mean  std  ...       50%       75%       max
key1 key2                        ...                              
a    1       1.0 -0.204708  NaN  ... -0.204708 -0.204708 -0.204708
     2       1.0  0.478943  NaN  ...  0.478943  0.478943  0.478943
b    1       1.0  1.965781  NaN  ...  1.965781  1.965781  1.965781
     2       1.0 -0.555730  NaN  ... -0.555730 -0.555730 -0.555730

[4 rows x 8 columns]

In [15]:
grouped2["data2"].describe().round(6)

count      mean  std  ...       50%       75%       max
key1 key2                        ...                              
a    1       1.0  0.281746  NaN  ...  0.281746  0.281746  0.281746
     2       1.0  0.769023  NaN  ...  0.769023  0.769023  0.769023
b    1       1.0 -1.296221  NaN  ... -1.296221 -1.296221 -1.296221
     2       1.0  1.007189  NaN  ...  1.007189  1.007189  1.007189

[4 rows x 8 columns]

### Iterating over Groups

The object returned by `groupby` supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. 

In [16]:
for group_key, group_values in df.groupby("key1"):
    print("group name = ", group_key)
    print(group_values)

group name =  a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
group name =  b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [17]:
for group_key, group_values in df.groupby(["key1", "key2"]):
    print("group name = ", group_key)
    print(group_values)  

group name =  ('a', 1)
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
group name =  ('a', 2)
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
group name =  ('b', 1)
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
group name =  ('b', 2)
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


### Selecting a Column or Subset a Column

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.

In [18]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [19]:
# group_by `key1` column and selecting `data1` column
key1_data1 = df.groupby("key1")["data1"]

In [20]:
pd.DataFrame(key1_data1)

,0,1
0,a,0 -0.204708 1 0.478943 5 1.393406 Name...
1,b,"3 -0.555730 4 1.965781 Name: data1, dtype..."


In [21]:
key1_data1.sum()

key1
a    1.667642
b    1.410050
Name: data1, dtype: float64

In [22]:
# group_by `key1` column and selecting `data1` column
key1_data2 = df.groupby("key1")["data2"]
pd.DataFrame(key1_data2)

,0,1
0,a,0 0.281746 1 0.769023 5 0.274992 Name...
1,b,"3 1.007189 4 -1.296221 Name: data2, dtype..."


In [23]:
key1_data2.describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.441920,0.283299,0.274992,0.278369,0.281746,0.525384,0.769023
b,2.0,-0.144516,1.628757,-1.296221,-0.720368,-0.144516,0.431337,1.007189


### Grouping with Dictionaries and Series

In [24]:
np.random.seed(12345)
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.204708,0.478943,-0.519439,-0.555730,1.965781
Steve,1.393406,0.092908,0.281746,0.769023,1.246435
Wanda,1.007189,NaN,NaN,0.228913,1.352917
Jill,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Trey,-0.539741,0.476985,3.248944,-1.021228,-0.577087


In [25]:
people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, Joe to Trey
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       5 non-null      float64
 1   b       4 non-null      float64
 2   c       4 non-null      float64
 3   d       5 non-null      float64
 4   e       5 non-null      float64
dtypes: float64(5)
memory usage: 412.0+ bytes


In [26]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [27]:
by_column = people.groupby(mapping, axis="columns")
pd.DataFrame(by_column)

,0,1
0,blue,c d Joe -0.519439 -0.5...
1,red,a b e Joe -0.2...


In [28]:
pd.DataFrame(by_column).loc[0][1]

,c,d
Joe,-0.519439,-0.555730
Steve,0.281746,0.769023
Wanda,NaN,0.228913
Jill,-0.371843,1.669025
Trey,3.248944,-1.021228


In [29]:
# blue column
pd.DataFrame(by_column).loc[0][1].sum(axis=1)

Joe     -1.075169
Steve    1.050769
Wanda    0.228913
Jill     1.297183
Trey     2.227716
dtype: float64

In [30]:
# red column
pd.DataFrame(by_column).loc[1][1].sum(axis=1)

Joe      2.240016
Steve    2.732748
Wanda    2.360106
Jill    -1.553778
Trey    -0.639844
dtype: float64

In [31]:
by_column.sum()

,blue,red
Joe,-1.075169,2.240016
Steve,1.050769,2.732748
Wanda,0.228913,2.360106
Jill,1.297183,-1.553778
Trey,2.227716,-0.639844


In [32]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [33]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [34]:
people.round(4)

,a,b,c,d,e
Joe,-0.2047,0.4789,-0.5194,-0.5557,1.9658
Steve,1.3934,0.0929,0.2817,0.7690,1.2464
Wanda,1.0072,NaN,NaN,0.2289,1.3529
Jill,0.8864,-2.0016,-0.3718,1.6690,-0.4386
Trey,-0.5397,0.4770,3.2489,-1.0212,-0.5771


In [35]:
# grouping by name length
people.groupby(len)["a"].describe().round(4)

,count,mean,std,min,25%,50%,75%,max
3,1.0,-0.2047,NaN,-0.2047,-0.2047,-0.2047,-0.2047,-0.2047
4,2.0,0.1733,1.0085,-0.5397,-0.1832,0.1733,0.5299,0.8864
5,2.0,1.2003,0.2731,1.0072,1.1037,1.2003,1.2969,1.3934


In [36]:
people.groupby(len, axis=0).max()

,a,b,c,d,e
3,-0.204708,0.478943,-0.519439,-0.555730,1.965781
4,0.886429,0.476985,3.248944,1.669025,-0.438570
5,1.393406,0.092908,0.281746,0.769023,1.352917


In [37]:
people2 = people.copy()
people2.columns = ['aa','b', 'cc', 'd','e']
people2

,aa,b,cc,d,e
Joe,-0.204708,0.478943,-0.519439,-0.555730,1.965781
Steve,1.393406,0.092908,0.281746,0.769023,1.246435
Wanda,1.007189,NaN,NaN,0.228913,1.352917
Jill,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Trey,-0.539741,0.476985,3.248944,-1.021228,-0.577087


In [38]:
people2.groupby(len, axis=1).max()

,1,2
Joe,1.965781,-0.204708
Steve,1.246435,1.393406
Wanda,1.352917,1.007189
Jill,1.669025,0.886429
Trey,0.476985,3.248944


In [39]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list],axis=0).min()

,,a,b,c,d,e
3,one,-0.204708,0.478943,-0.519439,-0.555730,1.965781
4,two,-0.539741,-2.001637,-0.371843,-1.021228,-0.577087
5,one,1.007189,0.092908,0.281746,0.228913,1.246435


In [40]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list],axis=1).min()

1          
            one       two
Joe   -0.519439 -0.555730
Steve  0.092908  0.769023
Wanda  1.007189  0.228913
Jill  -2.001637 -0.438570
Trey  -0.539741 -1.021228

### Grouping by Index Levels

> The ability to aggregate using one of the levels of an axis index. 

In [41]:
np.random.seed(12345)
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.204708  0.478943 -0.519439 -0.555730  1.965781
1      1.393406  0.092908  0.281746  0.769023  1.246435
2      1.007189 -1.296221  0.274992  0.228913  1.352917
3      0.886429 -2.001637 -0.371843  1.669025 -0.438570

In [42]:
hier_df2 = hier_df.groupby(level="cty", axis="columns")
pd.DataFrame(hier_df2)

,0,1
0,JP,cty JP tenor 1 ...
1,US,cty US tenor ...


In [43]:
pd.DataFrame(hier_df2)[1][0]

cty          JP          
tenor         1         3
0     -0.555730  1.965781
1      0.769023  1.246435
2      0.228913  1.352917
3      1.669025 -0.438570

In [44]:
pd.DataFrame(hier_df2)[1][1]

cty          US                    
tenor         1         3         5
0     -0.204708  0.478943 -0.519439
1      1.393406  0.092908  0.281746
2      1.007189 -1.296221  0.274992
3      0.886429 -2.001637 -0.371843

In [45]:
hier_df2.count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [46]:
pd.DataFrame(hier_df2)[1][1]

cty          US                    
tenor         1         3         5
0     -0.204708  0.478943 -0.519439
1      1.393406  0.092908  0.281746
2      1.007189 -1.296221  0.274992
3      0.886429 -2.001637 -0.371843

In [47]:
hier_df.groupby(level="cty", axis="columns").count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2.3.2 Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays. For example, `mean`, `count`, `min`, and `sum`.

In [48]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [49]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(1)

key1   
a     0   -0.204708
b     3   -0.555730
Name: data1, dtype: float64

In [50]:
# Using a user-defined aggregation
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


In [51]:
grouped.describe().round(4).transpose()

key1              a       b
key2  count     2.0     2.0
      mean      1.5     1.5
      std    0.7071  0.7071
      min       1.0     1.0
      25%      1.25    1.25
      50%       1.5     1.5
      75%      1.75    1.75
      max       2.0     2.0
data1 count     3.0     2.0
      mean   0.5559   0.705
      std    0.8018   1.783
      min   -0.2047 -0.5557
      25%    0.1371  0.0746
      50%    0.4789   0.705
      75%    0.9362  1.3354
      max    1.3934  1.9658
data2 count     3.0     2.0
      mean   0.4419 -0.1445
      std    0.2833  1.6288
      min     0.275 -1.2962
      25%    0.2784 -0.7204
      50%    0.2817 -0.1445
      75%    0.5254  0.4313
      max     0.769  1.0072

In [52]:
grouped.describe().round(6)["key2"]

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0
b,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [53]:
grouped.describe().round(6)["data1"]

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.555881,0.801830,-0.204708,0.137118,0.478943,0.936175,1.393406
b,2.0,0.705025,1.782977,-0.555730,0.074647,0.705025,1.335403,1.965781


In [54]:
grouped.describe().round(6)["data2"]

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.441920,0.283299,0.274992,0.278369,0.281746,0.525384,0.769023
b,2.0,-0.144516,1.628757,-1.296221,-0.720368,-0.144516,0.431337,1.007189


### Column-Wise and Multiple Function Application

In [55]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [56]:
### Adding a new column
tips["tip_pct"] = (tips["tip"] / tips["total_bill"])
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [57]:
tips_grouped =  tips.groupby(["day", "smoker"])
pd.DataFrame(tips_grouped)

,0,1
0,"(Fri, No)",total_bill tip smoker day time siz...
1,"(Fri, Yes)",total_bill tip smoker day time siz...
2,"(Sat, No)",total_bill tip smoker day time siz...
3,"(Sat, Yes)",total_bill tip smoker day time si...
4,"(Sun, No)",total_bill tip smoker day time siz...
5,"(Sun, Yes)",total_bill tip smoker day time siz...
6,"(Thur, No)",total_bill tip smoker day time si...
7,"(Thur, Yes)",total_bill tip smoker day time siz...


In [58]:
pd.DataFrame(tips_grouped)[1][0]

,total_bill,tip,smoker,day,time,size,tip_pct
91,22.49,3.50,No,Fri,Dinner,2,0.155625
94,22.75,3.25,No,Fri,Dinner,2,0.142857
99,12.46,1.50,No,Fri,Dinner,2,0.120385
223,15.98,3.00,No,Fri,Lunch,3,0.187735


In [59]:
tips_grouped_pct = tips_grouped["tip_pct"]
tips_grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [60]:
pd.DataFrame(tips_grouped)[1][1]

,total_bill,tip,smoker,day,time,size,tip_pct
90,28.97,3.00,Yes,Fri,Dinner,2,0.103555
92,5.75,1.00,Yes,Fri,Dinner,2,0.173913
93,16.32,4.30,Yes,Fri,Dinner,2,0.263480
95,40.17,4.73,Yes,Fri,Dinner,4,0.117750
96,27.28,4.00,Yes,Fri,Dinner,2,0.146628
97,12.03,1.50,Yes,Fri,Dinner,2,0.124688
98,21.01,3.00,Yes,Fri,Dinner,2,0.142789
100,11.35,2.50,Yes,Fri,Dinner,2,0.220264
101,15.38,3.00,Yes,Fri,Dinner,2,0.195059
220,12.16,2.20,Yes,Fri,Lunch,2,0.180921


In [61]:
tips_grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [62]:
# Selecting and renaming a column 
tips_grouped_pct.agg([("average", "mean"), ("stdev", "std")])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [63]:
functions = ["count", "mean", "max"]
result = tips_grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [64]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [65]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
tips_grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [66]:
tips_grouped.agg({"tip" : np.max, "size" : "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [67]:
tips_grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [68]:
tips_grouped = tips.groupby(["day", "smoker"], as_index=False)
tips_grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 2.3.3  Apply: General split-apply-combine

The most general-purpose GroupBy method is `apply`. 
`apply` splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces.

In [69]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535


In [70]:
tips.groupby("smoker").apply(top).round(3)

total_bill   tip smoker   day    time  size  tip_pct
smoker                                                          
No     232       11.61  3.39     No   Sat  Dinner     2    0.292
       149        7.51  2.00     No  Thur   Lunch     2    0.266
       51        10.29  2.60     No   Sun  Dinner     2    0.253
       185       20.69  5.00     No   Sun  Dinner     5    0.242
       88        24.71  5.85     No  Thur   Lunch     2    0.237
Yes    172        7.25  5.15    Yes   Sun  Dinner     2    0.710
       178        9.60  4.00    Yes   Sun  Dinner     2    0.417
       67         3.07  1.00    Yes   Sat  Dinner     1    0.326
       183       23.17  6.50    Yes   Sun  Dinner     4    0.281
       109       14.31  4.00    Yes   Sat  Dinner     2    0.280

In [71]:
tips.groupby("day").apply(top).round(3)

total_bill   tip smoker   day    time  size  tip_pct
day                                                           
Fri  93        16.32  4.30    Yes   Fri  Dinner     2    0.263
     221       13.42  3.48    Yes   Fri   Lunch     2    0.259
     222        8.58  1.92    Yes   Fri   Lunch     1    0.224
     100       11.35  2.50    Yes   Fri  Dinner     2    0.220
     226       10.09  2.00    Yes   Fri   Lunch     2    0.198
Sat  67         3.07  1.00    Yes   Sat  Dinner     1    0.326
     232       11.61  3.39     No   Sat  Dinner     2    0.292
     109       14.31  4.00    Yes   Sat  Dinner     2    0.280
     214       28.17  6.50    Yes   Sat  Dinner     3    0.231
     20        17.92  4.08     No   Sat  Dinner     2    0.228
Sun  172        7.25  5.15    Yes   Sun  Dinner     2    0.710
     178        9.60  4.00    Yes   Sun  Dinner     2    0.417
     183       23.17  6.50    Yes   Sun  Dinner     4    0.281
     51        10.29  2.60     No   Sun  Dinner     2    0.253
     181       23.33  5.65    Yes   Sun  Dinner     2    0.242
Thur 149        7.51  2.00     No  Thur   Lunch     2    0.266
     194       16.58  4.00    Yes  Thur   Lunch     2    0.241
     88        24.71  5.85     No  Thur   Lunch     2    0.237
     87        18.28  4.00     No  Thur   Lunch     2    0.219
     200       18.71  4.00    Yes  Thur   Lunch     3    0.214

In [72]:
tips.groupby("time").apply(top).round(3)

total_bill   tip smoker   day    time  size  tip_pct
time                                                            
Dinner 172        7.25  5.15    Yes   Sun  Dinner     2    0.710
       178        9.60  4.00    Yes   Sun  Dinner     2    0.417
       67         3.07  1.00    Yes   Sat  Dinner     1    0.326
       232       11.61  3.39     No   Sat  Dinner     2    0.292
       183       23.17  6.50    Yes   Sun  Dinner     4    0.281
Lunch  149        7.51  2.00     No  Thur   Lunch     2    0.266
       221       13.42  3.48    Yes   Fri   Lunch     2    0.259
       194       16.58  4.00    Yes  Thur   Lunch     2    0.241
       88        24.71  5.85     No  Thur   Lunch     2    0.237
       222        8.58  1.92    Yes   Fri   Lunch     1    0.224

In [73]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [74]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [75]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the Group Keys

In [76]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile and Bucket Analysis

In [77]:
### Slicing data up into buckets with bins  or by sample quantiles
np.random.seed(123456)
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.469112,-0.218470
1,-0.282863,-0.061645
2,-1.509059,-0.723780
3,-1.135632,0.551225
4,1.212112,-0.497767


In [78]:
frame.shape

(1000, 2)

In [79]:
frame.describe()

,data1,data2
count,1000.000000,1000.000000
mean,-0.046044,0.013064
std,1.044086,1.000619
min,-3.520876,-2.927426
25%,-0.796742,-0.649535
50%,0.034666,-0.012551
75%,0.641440,0.698679
max,3.357427,3.565769


In [80]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0     (-0.0817, 1.638]
1    (-1.801, -0.0817]
2    (-1.801, -0.0817]
3    (-1.801, -0.0817]
4     (-0.0817, 1.638]
5    (-1.801, -0.0817]
6     (-0.0817, 1.638]
7    (-1.801, -0.0817]
8    (-1.801, -0.0817]
9     (-3.528, -1.801]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.528, -1.801] < (-1.801, -0.0817] < (-0.0817, 1.638] < (1.638, 3.357]]

In [81]:
np.unique(quartiles)

array([Interval(-3.528, -1.801, closed='right'),
       Interval(-1.801, -0.0817, closed='right'),
       Interval(-0.0817, 1.638, closed='right'),
       Interval(1.638, 3.357, closed='right')], dtype=object)

In [82]:
quartiles.value_counts()

(-0.0817, 1.638]     505
(-1.801, -0.0817]    397
(-3.528, -1.801]      50
(1.638, 3.357]        48
Name: data1, dtype: int64

In [83]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

quartiles_grouped = frame.groupby(quartiles)
quartiles_grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-3.528, -1.801]  data1 -3.520876 -1.828740     50 -2.257520
                  data2 -2.621377  2.133239     50 -0.030640
(-1.801, -0.0817] data1 -1.788308 -0.082240    397 -0.814450
                  data2 -2.553831  3.565769    397 -0.000555
(-0.0817, 1.638]  data1 -0.079915  1.626369    505  0.576472
                  data2 -2.927426  3.124365    505  0.011155
(1.638, 3.357]    data1  1.643563  3.357427     48  2.063547
                  data2 -2.833091  2.543834     48  0.191319

In [84]:
quartiles_grouped.agg(["min", "max", "count", "mean"])

data1                  ...     data2                
                        min       max count  ...       max count      mean
data1                                        ...                          
(-3.528, -1.801]  -3.520876 -1.828740    50  ...  2.133239    50 -0.030640
(-1.801, -0.0817] -1.788308 -0.082240   397  ...  3.565769   397 -0.000555
(-0.0817, 1.638]  -0.079915  1.626369   505  ...  3.124365   505  0.011155
(1.638, 3.357]     1.643563  3.357427    48  ...  2.543834    48  0.191319

[4 rows x 8 columns]

In [85]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    1
2    0
3    0
4    3
Name: data1, dtype: int64

In [86]:
quartiles_grouped = frame.groupby(quartiles_samp)
quartiles_grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.520876 -0.798334    250 -1.412055
      data2 -2.621377  3.160915    250 -0.020340
1     data1 -0.796211  0.033113    250 -0.339153
      data2 -2.038905  3.565769    250  0.076206
2     data1  0.036220  0.641385    250  0.322669
      data2 -2.927426  3.124365    250 -0.061915
3     data1  0.641606  3.357427    250  1.244361
      data2 -2.833091  2.543834    250  0.058306

### Example: Filling Missing Values with Group-Specific Value

In [87]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    0.346823
2         NaN
3    1.185464
4         NaN
5    0.438302
dtype: float64

In [89]:
#  filling the null values with the mean
s.fillna(s.mean())

0    0.656863
1    0.346823
2    0.656863
3    1.185464
4    0.656863
5    0.438302
dtype: float64

**Filling the values that vary with groups**

In [92]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          1.020395
New York     -0.584339
Vermont      -0.504424
Florida      -1.091023
Oregon       -0.224136
Nevada       -0.379561
California    0.682261
Idaho         0.475287
dtype: float64

In [93]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          1.020395
New York     -0.584339
Vermont            NaN
Florida      -1.091023
Oregon       -0.224136
Nevada             NaN
California    0.682261
Idaho              NaN
dtype: float64

In [97]:
pd.DataFrame(data.groupby(group_key))

,0,1
0,East,Ohio 1.020395 New York -0.584339 Verm...
1,West,Oregon -0.224136 Nevada NaN ...


In [94]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [96]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [98]:
data.groupby(group_key).mean()

East   -0.218322
West    0.229063
dtype: float64

In [99]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          1.020395
New York     -0.584339
Vermont      -0.218322
Florida      -1.091023
Oregon       -0.224136
Nevada        0.229063
California    0.682261
Idaho         0.229063
dtype: float64

In [100]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

Ohio          1.020395
New York     -0.584339
Vermont       0.500000
Florida      -1.091023
Oregon       -0.224136
Nevada       -1.000000
California    0.682261
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [101]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [102]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [106]:
np.random.seed(123456)
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

6D      6
4S      4
10C    10
10H    10
3D      3
dtype: int64

In [107]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  4C     4
   7C     7
D  7D     7
   8D     8
H  5H     5
   KH    10
S  QS    10
   3S     3
dtype: int64

In [108]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

QC    10
AC     1
AD     1
2D     2
QH    10
KH    10
8S     8
KS    10
dtype: int64

### Example: Group Weighted Average and Correlation

In [113]:
np.random.seed(12345)
df2 = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df2

,category,data,weights
0,a,-0.204708,0.748907
1,a,0.478943,0.653570
2,a,-0.519439,0.747715
3,a,-0.555730,0.961307
4,b,1.965781,0.008388
5,b,1.393406,0.106444
6,b,0.092908,0.298704
7,b,0.281746,0.656411


In [115]:
wac_grouped = df2.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

wac_grouped.apply(get_wavg)

category
a   -0.245188
b    0.352824
dtype: float64

In [118]:
# Another example
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [119]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [120]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [121]:
rets = close_px.pct_change().dropna()
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [122]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [123]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [124]:
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

You can install `statsmodels` with conda if you don't have it already:

```
conda install statsmodels
```

In [125]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 2.3.4 Group Transforms and "Unwrapped" GroupBys

**THIS IS A READING ASSIGNMENT**

## 2.3.5  Pivot Tables and Cross-Tabulation
**THIS IS A READING ASSIGNMENT**